# Preparar entorno

## Instalar Librerias

### Instalar librerias para sugerencias de codigo

In [ ]:
!pip install jupyter_contrib_nbextensions
!pip install jupyter_nbextensions_configurator
!jupyter contrib nbextension install --user

In [ ]:
!pip install langchain

In [ ]:
!pip install python-dotenv

In [ ]:
from dotenv import load_dotenv
import os


In [ ]:
load_dotenv()

In [ ]:
print(os.environ["OPENAI_API_KEY"])

In [ ]:
print(os.environ["GOOGLE_API_KEY"])

# Prompt Template

In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt = PromptTemplate.from_template("Describe un objeto que te resulte {adjetivo} y por qué tiene ese efecto en tí")

In [ ]:
prompt.format(adjetivo="fascinante")

In [ ]:
prompt.format(adjetivo="feo")

## Cadenas con propmt template

In [ ]:
template = "Eres un asistente util que traduce del {idioma_entrada} al {idioma_salida} el texto {texto}"

In [ ]:
template

In [ ]:
texto = "Me encanta programar"

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["idioma_entrada","idioma_salida","texto"],template=template
)

In [ ]:
!pip install langchain-openai

### Llamando al LLM

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(
    temperature = 1,
    model_name = "gpt-4.1-2025-04-14"
)

In [ ]:
## Debido a que no hay creditos disponibles de OPENIA, se usara un modelo de GeminAI

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    #model="gemini-1.5-pro",
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
chain = prompt_template | llm

In [ ]:
respuesta = chain.invoke(input={"idioma_entrada":"español","idioma_salida":"francés","texto":texto})
print(respuesta)

### Obtener informacion de peliculas (Sin langchain)

In [ ]:
import os
import requests
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

In [ ]:
!pip install requests

In [ ]:
import requests
pelicula = "titanic"
#url = "https://api.themoviedb.org/3/authentication"
url = f"https://api.themoviedb.org/3/search/movie?query={pelicula}&include_adult=false&language=en-US&page=1"

headers = {
    "accept": "application/json",
    "Authorization": "Bearer *******"
}

response = requests.get(url, headers=headers)

respuesta = response.json()
respuesta

In [ ]:
resumen = respuesta["results"][0]["overview"]
print(resumen)

In [ ]:
titulo = respuesta["results"][0]["original_title"]
print(titulo)

In [ ]:
fecha = respuesta["results"][0]["release_date"]
print(fecha)

In [ ]:
print(len(respuesta["results"]))

In [ ]:
for i in range(min(len(respuesta["results"]),3)):
    titulo = respuesta["results"][i]["original_title"]
    resumen = respuesta["results"][i]["overview"]
    fecha = respuesta["results"][i]["release_date"]
    print(f""" Titulo: {titulo}
    Fecha de entreno: {fecha}
    Resumen: {resumen}
    """)

### Obtener información de peliculas con langchain

In [ ]:
# Envolver la conexion al servicio en un metodo de python
import requests
def peliculas(pelicula):
    url = f"https://api.themoviedb.org/3/search/movie?query={pelicula}&include_adult=false&language=en-US&page=1"
    headers = {
        "accept": "application/json",
        "Authorization": "Bearer ******"
    }
    response = requests.get(url, headers=headers)
    return response

In [ ]:
template = """Te voy a dar informacion sobre algunas peliculas, me tienes que dar la informacion (en español) 
del titulo, fecha de estreno y resumen de las primeras 3 que aparezcan de forma estructurada (si aparecen menos, me das las que aparezcan).
{respuesta}
"""

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["respuesta"], template=template
)


In [ ]:
# Ahora llamamos al modelo de IA. Usamos GeminIA porque OpenIA es de paga
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    #model="gemini-1.5-pro",
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template)

In [ ]:
respuesta=peliculas("titanic")

In [ ]:
print(chain.run(respuesta=respuesta.text))
#print(chain.invoke({"respuesta": respuesta.text}))

# Cadenas Simples

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    #model="gemini-1.5-pro",
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
template = """Eres un detective experimentado. Describe las pistas clave que condujeron a resolver el caso de 
{caso} en {ciudad}. """

In [ ]:
prompt_template = PromptTemplate.from_template(template=template)

In [ ]:
chain = LLMChain(
    llm = llm,
    prompt = prompt_template,
    verbose = True  
)

In [ ]:
output = chain.invoke({'caso':'Desaparicion de la madrastra','ciudad':'Londres'})

In [ ]:
print(output)

In [ ]:
print(output["text"])

## Cadenas secuenciales

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm1 = ChatGoogleGenerativeAI(
    #model="gemini-1.5-pro",
    model="gemini-2.0-flash",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
template1 = """Eres un científico experimentado y programador en Python. 
Escriba una funcion que implemente el concepto de {concepto}"""

In [ ]:
prompt_template1 = PromptTemplate.from_template(
    template="Eres un científico experimentado y programador en Python. Escriba una funcion que implemente el concepto de {concepto}."
)

In [ ]:
chain1 = LLMChain(
    llm = llm1,
    prompt = prompt_template1,
    verbose = True  
)

In [ ]:
# Ahora creamos un modelo 2
llm2 = ChatGoogleGenerativeAI(
    #model="gemini-1.5-pro",
    model="gemini-2.0-flash",
    temperature=1.2,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
prompt_template2 = PromptTemplate.from_template(
    template="Dada la funcion Python {funcion}. Descríbela lo más detalladamente posible."
)

In [ ]:
chain2 = LLMChain(
    llm = llm2,
    prompt = prompt_template2,
    verbose = True  
)

In [ ]:
# Vamos a crear una cadena secuencial
overall_chain = SimpleSequentialChain(chains=[chain1,chain2], verbose=True)

In [ ]:
# Ahora le parametrizamos el concepto 'obtener los primeros "n" números primos'
output = overall_chain.invoke('obtener los primeros "n" números primos')

## ReAct Agent (RAG)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.1,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
output = llm.invoke('explica el procesamiento de lenguaje natural en una oración')
print(output.content)

In [ ]:
from langchain.schema import(
    SystemMessage,
    AIMessage,
    HumanMessage
)

In [ ]:
messages = [
    SystemMessage(content='Eres un chef y respondes solo con conceptos culinarios'),
    HumanMessage(content='explica procesamiento del lenguaje natural en una oración')
]

In [ ]:
# Le entregamos los messages y ejecutamos
output = llm.invoke(messages)
print(output.content)

## Agentes en acción

In [ ]:
!pip install langchain_experimental -q

In [ ]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(0,100) if n%13 == 0])')

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
# Entre mas cerca a cero esté la temperatura del modelo, más determinístico; y entre más cerca a dos, el modelo es mas creativo
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
prompt = 'Encuentra el promedio de los cubos de los numeros del 1 al 10 y fuerza a que se vean 3 decimales'
respuesta = agent_executor.invoke(prompt)

In [ ]:
print(respuesta['input'])

In [ ]:
print(respuesta['output'])

## LangChain Tools: DuckDuckGo and Wikipedia

### DuckDuckGo

In [ ]:
!pip install duckduckgo-search

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

In [ ]:
search = DuckDuckGoSearchRun()

In [ ]:
output = search.invoke('¿Cuál es el principal ingrediente de la pizza Margarita?')
print(output)

### Otra forma de usar esta libreria (con método run)

In [ ]:
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
output = search.run('¿Cuál es el principal ingrediente de la pizza Margarita?')
print(output)

### Ahora con un Wrapper

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

In [ ]:
wrapper = DuckDuckGoSearchAPIWrapper(region='co-es', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('nvidia')

In [ ]:
print(output)

In [ ]:
output = search.run('William Enrique Parra Alba')
print(output)

#### Se puede dar un poco mas de formato a la salida

In [ ]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)\],'
matches = re.findall(pattern, output, re.DOTALL)
#print(matches)
for snippet, title, link in matches:
    print(f'Snippet: {snippet}\nTitle: {title}\bnLink: {link}\n')
    print('-' * 50)


## Wikipedia

In [ ]:
!pip install wikipedia

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper(lang='es', top_k_results=1, doc_content_chars_max=10000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
# Que busque sora de OpenAI
wiki.invoke({'query':'sora de openai'})

## Crear ReAct Agent

In [ ]:
!pip install langchainhub

In [ ]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
#from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
# Creamos el modelo
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
template = '''
Responde las siguientes preguntas en Italiano lo mejor que puedas.
Preguntas: {q}
'''

In [ ]:
prompt_template = PromptTemplate.from_template(template)

In [ ]:
prompt = hub.pull('hwchase17/react')
"""
Este metodo te permite extraer y utilizar objetos, como agentes, configuraciones o plantillas,
    que han sido compartidos en LangChain Hub por otros usuarios o creadores o default de langchain. En este caso particular
    'hwchase17/react' hace referencia a una configuración o componente específico, que está relacionado con la funcionalidad.

    Esta funcionalidad es particularmente útil cuando deseas implementar lógicas complejas o específicas sin
    tener que desarrollar todo desde cero. Al aprovechar los componentes compartidos en LangChain Hub, puedes
    enriquecer tus aplicaciones con funcionalidades avanzadas, facilitando el desarrollo de soluciones basadas
    en modelos de lenguaje grande (LLM) y otras herramientas de procesamiento de lenguaje natural (NLP).
"""

In [ ]:
# 1. Python REPL Tool
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name = 'Python REPL',
    func=python_repl.run,
    description='Útil cuando necesitas usar Python para responder una pregunta. Debes ingresar código Python.'
)

In [ ]:
# 2. Wikipedia tool
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Útil cuando necesitas buscar un tema, país o persona en Wikipedia.'
)

In [ ]:
# 3. DuckDuckGo Search Tool
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Útil cuando necesitas realizar una búsqueda en internet para encontrar información que otra herramienta no puede proporcionar.'
)

In [ ]:
# Se genera una lista con las herramientas
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

In [ ]:
# Se crea el agente
agent = create_react_agent(llm, tools, prompt)

In [ ]:
# Se ejecuta el agente
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

In [ ]:
# Generamos la consulta al agente
question = 'Cuéntame sobre la vida temprana de Simón Bolívar'
output = agent_executor.invoke({
    'input':prompt_template.format(q=question)
})

In [ ]:
# Se pinta la respuesta en italiano
output['output']

In [ ]:
# Ahora otra pregunta:
question = '¿Qué país ganó la última copa del mundo de fútbol?'
output = agent_executor.invoke({
    'input':prompt_template.format(q=question)
})

In [ ]:
# Se pinta la respuesta en italiano
output['output']

In [ ]:
# Ahora esta otra pregunta:
question = 'Determina los primeros 100 números primos'
output = agent_executor.invoke({
    'input':prompt_template.format(q=question)
})

In [ ]:
# Imprime la respuesta
output['output']

## LangChain y Vector Stores (Pinecone)

In [ ]:
# Datos con base en vectores. Permite transformar datos de entrada en vectores y guardarlos para posteriores búsquedas por similitud.
!pip install pinecone-client

In [ ]:
!pip install --upgrade pinecone-client

In [ ]:
!pip show pinecone-client

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

In [ ]:
load_dotenv(find_dotenv(), override=True)

In [ ]:
print(os.environ["PINECONE_API_KEY"])

In [ ]:
!pip install pinecone

In [ ]:
!pip install --upgrade pinecone

In [ ]:
!pip show pinecone

In [ ]:
from pinecone import Pinecone

In [ ]:
pc = Pinecone()

In [ ]:
pc.list_indexes()

In [ ]:
index_name = 'langchain'
pc.describe_index(index_name)

In [ ]:
# Nombre de todos los indices
pc.list_indexes().names()

In [ ]:
# Podemos borra el indice de Pinecone
# pc.delete_index(index_name)

In [ ]:
pc.list_indexes()

In [ ]:
# Crear un indice desde el código
#from pinecone import PodSpec
#index_name = 'langchain2'
#pc.create_index(
#    name=index_name,
#    dimension=3072,
#    metric='cosine',
#    spec=PodSpec(
#        environment='gcp-starter'
#    )
#)
print("echo")

In [ ]:
index = pc.Index(index_name)
#index = pc.list_indexes().names()[0]
print(index)

In [ ]:
index.describe_index_stats()

In [ ]:
# Ahora que ya está el indice, se van a crear los vectores
import random

In [ ]:
vectors = [[random.random() for _ in range(3072)] for v in range(5)]

In [ ]:
ids = list('abcde')

In [ ]:
ids

In [ ]:
index_name = 'langchain'

In [ ]:
index = pc.Index(index_name)

In [ ]:
# Guarda los vectores en el index langchain
index.upsert(vectors=zip(ids, vectors))

In [ ]:
# Se actualiza el vector C
index.upsert(vectors=[('c',[0.5] * 3072)])

In [ ]:
# Recuperamos los vectores
index.fetch(ids=['c','d'])

In [ ]:
# Eliminar vectores b y c
index.delete(ids=['b','c'])

In [ ]:
index.describe_index_stats()

In [ ]:
# Se hace una busqueda
query_vector = [random.random() for _ in range(3072)]
query_vector
print("")

In [ ]:
# Buscar los vectores que estan mas cerca al vector anterior
index.query(
    vector=query_vector,
    top_k=3,
    # Si se pone include_values=True, la salida incluye los valores de los vectores
    include_values=False
)

### Namespaces

In [ ]:
index = pc.Index('langchain')

In [ ]:
import random

In [ ]:
vectors = [[random.random() for _ in range(3072)] for v in range(5)]
ids = list('abcde')
index.upsert(vectors=zip(ids, vectors))

In [ ]:
# Ahora creamos tres vectores
vectors = [[random.random() for _ in range(3072)] for v in range(3)]
ids = list('xyz')
index.upsert(vectors=zip(ids, vectors),namespace='primer-namespace')

In [ ]:
# Ahora creamos dos vectores
vectors = [[random.random() for _ in range(3072)] for v in range(2)]
ids = list('qp')
index.upsert(vectors=zip(ids, vectors),namespace='segundo-namespace')

In [ ]:
index.describe_index_stats()

In [ ]:
# Recupero los valores de un namespace
index.fetch(ids=['x'],namespace='primer-namespace')
print("")

In [ ]:
# Puedo borrar un vector, pero para que sea efectivo el borrado, se debe incluir el namespace
index.delete(delete_all=True, namespace= 'primer-namespace')

In [ ]:
index.describe_index_stats()

## Uso de langchain y Pinecone

### Splitting y Embedding textos usando LangChain (Similarity Search)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
with open('La-Republilca-Platon.txt', encoding='utf-8') as f:
    hp7 = f.read()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=60,
    length_function=len
)

In [ ]:
chunks = text_splitter.create_documents([hp7])

In [ ]:
# Impime la salida
chunks
# Para no saturar la pantalla, imprimo algo pequeño.
#print("Algo")

In [ ]:
print(chunks[10].page_content)

In [ ]:
# Para ver cuántos fragmentos se tiene, se imprime el tamaño de los chunk.
print(f'Hay {len(chunks)} fragmentos')

## Crear Embedding
#### Pasamos los fragmentos en embeddings

In [ ]:
!pip install google

In [ ]:
!pip install google.genai

In [ ]:
# from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=3072)
# vector = embeddings.embed_query(chunks[0].page_content)
# vector

from google import genai
from google.genai import types

client = genai.Client()
text = chunks[0].page_content
embeddings = client.models.embed_content(
    model="text-embedding-004",
    contents=text,
    config=types.EmbedContentConfig(output_dimensionality=92),
)
print(result.embeddings)

### Insertamos este vectore en pinecone


In [ ]:
import os
import pinecone
from langchain_community.vectorstores import Pinecone

In [ ]:
pc = pinecone.Pinecone()

In [ ]:
# Borramos todos los indices
indexes = pc.list_indexes().names()
for i in indexes:
    print('Borrando los indices ...', end='')
    pc.delete_index(i)
    print('Listo..')

In [ ]:
# Se va a crear un indice con el contenido del libro
from pinecone import PodSpec
index_name = 'la-republica-platon'


In [ ]:
# Esta rutina no funciona porque la licencia no permite la creacion de indices desde el API
#if index_name not in pc.list_indexes().names():
#    print(f'Creando el indice {index_name}')
#    pc.create_index(
#        name=index_name,
#        dimension=3072,
#        metric='cosine',
#        spec=PodSpec(
#            environment='gcp-starter'
#        )
#    )
#    print('Indice Creado')
#else:
#    print(f'Indice {index_name} ya existe!')
   

In [ ]:
# Creacion de un indice
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

#pc = Pinecone(api_key="YOUR_API_KEY")
if index_name not in pc.list_indexes().names():
    print(f'Creando el indice {index_name}')
    pc.create_index(
      name=index_name,
      dimension=3072,
      metric="cosine",
      spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
      )
    )
    print('Indice Creado')
else:
    print(f'Indice {index_name} ya existe!')

In [ ]:
!pip install langchain_pinecone 

In [ ]:
#from langchain_community.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore.from_documents(
        chunks,
        index_name=index_name,
        embedding=embeddings
    )